<!--
This file is part of cjdk.
Copyright 2022 Board of Regents of the University of Wisconsin System
SPDX-License-Identifier: MIT
--->

# Running Java applications with jgo

This example shows how to run an arbitrary package available from a Maven
repository using [**jgo**](https://github.com/scijava/jgo).

In [1]:
import cjdk
import jgo
import os
from contextlib import contextmanager

**jgo** requires Apache Maven, so we'll install that first:

In [2]:
# From https://maven.apache.org/download.html
maven_url = "tgz+https://dlcdn.apache.org/maven/maven-3/3.8.6/binaries/apache-maven-3.8.6-bin.tar.gz"
maven_sha512 = "f790857f3b1f90ae8d16281f902c689e4f136ebe584aba45e4b1fa66c80cba826d3e0e52fdd04ed44b4c66f6d3fe3584a057c26dfcac544a60b301e6d0f91c26"

In [3]:
maven_dir = cjdk.cache_package("Maven", maven_url, sha512=maven_sha512)

cjdk: Installing Maven to /home/runner/.cache/cjdk


Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Extracting: 0files [00:00, ?files/s]

The Maven `.tar.gz` file has been extracted into `maven_dir`; find the `bin`
directory within it:

In [4]:
maven_bin = list(maven_dir.glob("apache-maven-*"))[0] / "bin"
assert (maven_bin / "mvn").is_file()

Let's write a context manager that we can use to temporarily put the Maven
`bin` directory on `PATH`.

In [5]:
@contextmanager
def path_prepended(path):
    """
    Context manager to temporarily prepend the given path to PATH.
    """
    save_path = os.environ.get("PATH", "")
    new_path = str(path) + os.pathsep + save_path
    os.environ["PATH"] = new_path
    try:
        yield
    finally:
        os.environ["PATH"] = save_path

Now for the magic: run a program by specifying its Maven coordinates.

The JRE and all required Jars are downloaded (and cached) automatically.

In [6]:
with cjdk.java_env(vendor="zulu-jre", version="8"):
    with path_prepended(maven_bin):
        jgo.main_from_endpoint(
            "com.puppycrawl.tools:checkstyle:9.3",
            primary_endpoint_main_class="com.puppycrawl.tools.checkstyle.Main",
            argv=["--version"],
        )

cjdk: Installing JDK zulu-jre:8.0.332 to /home/runner/.cache/cjdk


Downloading:   0%|          | 0.00/43.7M [00:00<?, ?B/s]

Extracting: 0files [00:00, ?files/s]

INFO 2022-07-05 15:39:37,159: First time start-up may be slow. Downloaded dependencies will be cached for shorter start-up times in subsequent executions.


Checkstyle version: 9.3
